In [1]:
from datetime import datetime
import os
import re

import pandas as pd
import polars as pl
import numpy as np
import matplotlib as plt
import seaborn as sns
import lightgbm as lgb
import sklearn as sk
import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [6]:
REMOTE_TRACKING_IP = os.getenv("REMOTE_IP", "localhost")
MLFLOW_TRACKING_URI = f"http://{REMOTE_TRACKING_IP}:5000"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

CREDIT_EXPERIMENT_NAME = "credit-score-train"
EXPERIMENT_NAME = "chosen-models-credit"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(CREDIT_EXPERIMENT_NAME)

2024/04/04 21:52:25 INFO mlflow.tracking.fluent: Experiment with name 'credit-score-train' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/667472224209157606', creation_time=1712260345326, experiment_id='667472224209157606', last_update_time=1712260345326, lifecycle_stage='active', name='credit-score-train', tags={}>

In [7]:
def get_splits(data: pl.DataFrame):
    case_ids = data[["case_id"]].unique()
    case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.7)
    case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5)
    return case_ids_train, case_ids_valid, case_ids_test

In [26]:
def from_polars_to_pandas(data, case_ids: pl.DataFrame, cat_cols=None) -> pd.DataFrame:
    base = data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas()
    X = data.filter(pl.col("case_id").is_in(case_ids))[[x for x in data.columns if x not in ["case_id", "WEEK_NUM", "target"]]].to_pandas()
    y = data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    if cat_cols is None:
        cat_cols = list(X.select_dtypes("object").columns)
    
    X[cat_cols] = X[cat_cols].astype("category")
    return base, X, y

In [27]:
def load_data_splits(data_path: str):
    data = pl.read_parquet(data_path, use_pyarrow = True)
    case_ids_train, case_ids_valid, case_ids_test = get_splits(data)
    return (
        from_polars_to_pandas(data, case_ids_train),
        from_polars_to_pandas(data, case_ids_valid),
        from_polars_to_pandas(data, case_ids_test)
    )

In [28]:
(base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = load_data_splits("data/train_base.parquet")
cat_cols_base = list(X_train.select_dtypes("category").columns)

In [56]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

In [57]:
def train_lgb_model(params, train_data, valid_data) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                        callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
        mlflow.log_metric("auc", eval_result['eval']['auc'][-1])
        return bst, eval_result

In [58]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}


bst, eval_results = train_lgb_model(params, train_data, valid_data)

Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.777952
[100]	eval's auc: 0.797256
[150]	eval's auc: 0.805721
[200]	eval's auc: 0.811403
[250]	eval's auc: 0.814745
[300]	eval's auc: 0.817435
[350]	eval's auc: 0.819253
[400]	eval's auc: 0.820796
[450]	eval's auc: 0.822087
[500]	eval's auc: 0.823132
[550]	eval's auc: 0.824052
[600]	eval's auc: 0.824725
[650]	eval's auc: 0.825416
[700]	eval's auc: 0.825934
Early stopping, best iteration is:
[736]	eval's auc: 0.826282


In [60]:
eval_results['eval']['auc'][-1]

0.8262716110570111

In [13]:
ypred = bst.predict(test_df[[x for x in test_df.columns if x != 'target']])

In [14]:
roc_auc_score(test_df['target'], ypred)

0.8303897059872279

In [21]:
f1_score(test_df['target'], ypred > 0.13)

0.22561422808947562

In [18]:
train_df['target']

0          0
1          0
2          0
3          0
4          1
          ..
1526654    0
1526655    0
1526656    0
1526657    0
1526658    0
Name: target, Length: 1526659, dtype: int64

In [17]:
np.percentile(ypred, 96)

0.13000668743092647

In [63]:
np.mean(train_df['target'])

0.03143711716845693

In [22]:
def get_lgbm_varimp(model, train_columns, max_vars=50):
    cv_varimp_df = pd.DataFrame([train_columns, model.feature_importance()]).T
    cv_varimp_df.columns = ['feature_name', 'varimp']
    cv_varimp_df.sort_values(by='varimp', ascending=False, inplace=True)
    cv_varimp_df = cv_varimp_df.iloc[0:max_vars]   
    return cv_varimp_df

In [23]:
imps = get_lgbm_varimp(bst, [x for x in train_df.columns if x != 'target'])

In [24]:
imps

,feature_name,varimp
0,case_id,117
36,annuity_780A,116
174,pmtnum_254L,114
178,price_1097A,113
290,min_cancelreason_3545846M,106
405,max_birth_259D,99
71,cntpmts24_3658933L,77
133,mobilephncnt_593L,74
434,max_incometype_1044T,62
86,eir_270L,62


In [57]:
test_data.get_label().shape

(305332,)

In [61]:
test_df.head()

,case_id,WEEK_NUM,target,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtcount_4527229L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,isbidproduct_1095L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_60

In [66]:
test_df['target'].head(10), ypred[:10]

(564688     0
 1342676    0
 740616     0
 995276     0
 1309958    0
 194279     0
 874456     0
 1221594    0
 450319     0
 736633     0
 Name: target, dtype: int64,
 array([0.01155312, 0.01155312, 0.01155312, 0.01155312, 0.01155312,
        0.01155312, 0.01155312, 0.01155312, 0.01155312, 0.01155312]))

In [73]:
test_df['target'].sum()

9599

In [72]:
np.unique(ypred, return_counts=True)

(array([0.01155312, 0.7784853 ]), array([295733,   9599]))

In [170]:
xdf = train_df[['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 
                 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 
                 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 
                 'inittransactionamount_650A','lastapprcredamount_781A', 'lastrejectcredamount_222A', 'maininc_215A', 'maxannuity_159A',
                 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A',
                 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'price_1097A', 
                 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 
                 'totinstallast1m_4525188A', 'description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 
                 'maritalst_893M', 'pmtaverage_3A', 'pmtaverage_4527227A', 'pmtssum_45A', 'target']]

In [171]:
tr_df, te_df = train_test_split(xdf, stratify=xdf['target'], test_size=0.2)
tr_df, va_df = train_test_split(tr_df, stratify=tr_df['target'], test_size=0.1)
trd = lgb.Dataset(tr_df[[x for x in tr_df.columns if x != 'target']], 
                  label=tr_df['target'], 
                  categorical_feature=['description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 'maritalst_893M']
                 )
ted = lgb.Dataset(te_df[[x for x in te_df.columns if x != 'target']], 
                  label=te_df['target'], 
                  categorical_feature=['description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 'maritalst_893M']
                 )
vad = lgb.Dataset(va_df[[x for x in va_df.columns if x != 'target']], 
                  label=va_df['target'], 
                  categorical_feature=['description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 'maritalst_893M']
                 )

In [172]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

chck = lgb.train(params, trd, valid_sets=vad, callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)])

/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.693382
[100]	valid_0's auc: 0.707502
[150]	valid_0's auc: 0.713174
[200]	valid_0's auc: 0.716931
[250]	valid_0's auc: 0.720201
[300]	valid_0's auc: 0.72282
[350]	valid_0's auc: 0.724252
[400]	valid_0's auc: 0.726079
[450]	valid_0's auc: 0.727473
[500]	valid_0's auc: 0.728876
[550]	valid_0's auc: 0.729794
[600]	valid_0's auc: 0.730611
[650]	valid_0's auc: 0.731657
[700]	valid_0's auc: 0.732023
[750]	valid_0's auc: 0.732569
Early stopping, best iteration is:
[764]	valid_0's auc: 0.73284


In [173]:
ypred = chck.predict(te_df[[x for x in te_df.columns if x != 'target']])

In [174]:
roc_auc_score(te_df['target'], ypred)

0.735416318264719

In [86]:
tr_df.iloc[:, :-1]

,dateofbirth_337D,education_1103M,credamount_770A,annuity_780A,price_1097A
1462243,-26397.0,a55475b1,70000.0,7953.4000,0.0
312003,-17372.0,a55475b1,27278.0,2273.2000,27278.0
978938,-15562.0,a55475b1,28898.0,2408.2000,38898.0
933185,-14129.0,a55475b1,27452.4,2287.8000,27452.4
631232,-24263.0,6b2ae0fa,5980.0,1184.6000,5980.0
...,...,...,...,...,...
929628,-17691.0,a55475b1,27650.0,1536.2001,27650.0
265955,NaN,a55475b1,17978.0,1665.8000,17978.0
38092,NaN,a55475b1,30000.0,3536.2000,NaN
1376314,-24366.0,6b2ae0fa,60000.0,11766.2000,0.0


In [84]:
from sklearn.linear_model import SGDClassifier

In [114]:
sgd = SGDClassifier(max_iter=1000, loss="log_loss", class_weight="balanced")

In [119]:
sgd.fit(tr_df[[x for x in tr_df.columns if x not in ['description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 
                 'maritalst_893M', 'target']]].fillna(0), tr_df['target'])

SGDClassifier(class_weight='balanced', loss='log_loss')

In [120]:
pred = sgd.predict(te_df[[x for x in tr_df.columns if x not in ['description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 
                 'maritalst_893M', 'target']]].fillna(0))

In [121]:
roc_auc_score(te_df['target'], pred)

0.5231452060506899

In [146]:
f1_score(te_df['target'], ypred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [124]:
accuracy_score(te_df['target'], pred)

0.8189924099137825